In [1]:
import os
import numpy as np
from pyrecode.recode_reader import ReCoDeReader
from collections import deque
from datetime import datetime
import time

%matplotlib inline
from matplotlib import pyplot as plt
from IPython.display import clear_output

In [2]:
class ReCoDeViewer():

    def __init__(self, folder_path, base_filename, num_parts, frames_per_sec, refresh_rate):
        
        self._num_parts = num_parts
        self._buffer_len = int(np.ceil((frames_per_sec*refresh_rate)/num_parts))
        self._refresh_rate = refresh_rate

        self._readers = []
        self._buffers = []
        self._pointers = []

        for index in range(num_parts):
            inter_file_name = os.path.join(folder_path, base_filename + '_part' + '{0:03d}'.format(index))
            recode_reader = ReCoDeReader(inter_file_name, is_intermediate=True)
            recode_reader.open(print_header=False)
            self._readers.append(recode_reader)
            d = deque(maxlen=self._buffer_len)
            self._buffers.append(d)
            self._pointers.append(0)
            
        header = self._readers[0].get_header().as_dict()
        self._ny = header['ny']
        self._nx = header['nx']

        self._view = None
        self._frame_start = 0
    
    def _get_latest_frames (self, part_index, is_first=False):
        '''get the last N frames, where N = self._buffer_len'''

        self._readers[part_index].open(print_header=False)
        if not is_first:
            self._readers[part_index]._fp.seek(self._pointers[part_index],0)
        
        f = self._readers[part_index].get_next_frame()
        while f is not None:
            if len(self._buffers[part_index]) == self._buffer_len:
                self._buffers[part_index].popleft()
            self._buffers[part_index].append(f)
            # self._change_trackers[part_index] = True
            f = self._readers[part_index].get_next_frame()
            
        self._pointers[part_index] = self._readers[part_index]._fp.tell()
        self._readers[part_index].close()
            
    def get_next_view (self):

        # start a refresh
        
        # note the time
        _t_start = datetime.now()
        
        # try get latest frames
        for index in range(self._num_parts):
            self._get_latest_frames(index)
        
        # merge frames
        frames = {}
        for part_index in range(self._num_parts):
            for f in self._buffers[part_index]:
                frame_id = list(f.keys())[0]
                frames[frame_id] = f[frame_id]['data']
        
        # if done earlier than refresh rate, wait
        _t_elapsed = (datetime.now() - _t_start).total_seconds()
        _wait_time = self._refresh_rate - _t_elapsed
        if _wait_time > 0.1:
            time.sleep(_wait_time)
            
        return frames
    
    def get_shape(self):
        return (self._ny, self._nx)

In [4]:
"""
_data_folder = '/scratch/loh/abhik/23-Jan-2020'
_dataset = 'gold_nano_1k_1k_dr=0.03'
_tag = 'gold_nano_1k_1k'
_num_part_files = 12
_frames_per_sec = 400
_refresh_rate = 1
t = 0
"""

_data_folder = '/home/abhik/code/ReCoDe/pyReCoDe/scratch'
_dataset = ''
_tag = 'run_1'
_num_part_files = 3
_frames_per_sec = 12
_refresh_rate = 1

viewer = ReCoDeViewer(os.path.join(_data_folder, _dataset), _tag + '.rc1', _num_part_files, _frames_per_sec, _refresh_rate)

time_step = 0
time_point = 0
view = np.zeros(viewer.get_shape(), dtype=np.uint16)
while time_step < 100:
    frames = viewer.get_next_view()
    for frame_id in frames:
        frame_data = frames[frame_id].todense()
        view = np.add(view, frame_data)
        
    n_frames = len(frames)
    frame_ids = list(frames.keys())
    min_frame = np.min(frame_ids)
    max_frame = np.max(frame_ids)
    
    fig, ax = plt.subplots(figsize=(10,10))
    im = ax.imshow(view, vmax=500)
    title_str = 'Time step (since start of viewer) = ' + str(time_step) + \
                ', Elapsed Time (since start of viewer) = ' + str(time_point) + \
                '\nShowing Sum of ' + str(n_frames) + ' Frames (from ' + str(min_frame) + ' to ' + str(max_frame) + ')'
    ax.set(title=title_str)
    # fig.colorbar(im)
    plt.show()
    clear_output(wait=True)
    
    time_step += 1
    time_point += _refresh_rate

KeyboardInterrupt: 